# chapter 2 자연어 처리 개발 준비

*** 1장은 파이썬 설치와 관련된 사항이기때문에 넘어가고 2장부터 시작해보자.***  
  
2번째 part 시작!

# 2. 사이킷런(Scikit-learn)

사이킷런은 파이썬용 머신러닝 라이브러리이다.  
사이킷런은 주로 지도학습, 비지도학습을 위한 모듈, 모델 선택과 평가를 위한 모듈, 데이터 변환 및 데이터를 불러오기 위한 모듈, 계산성능 향상을 위한 모듈로 구성돼있다.   
지도학습을 위한 모듈로는 나이브 베이즈, 의사결정 트리(Decision Trees), 서포트 벡터 머신(SVM)등  
비지도 학습을 위한 모듈에는 군집화(Clustering), 가우시안 혼합모델(Gaussian mixture models) 등이 있다.  
모델 선택과 평가모듈에는 교차검증(Cross Validation), 모델 평가, 모델의 지속성을 위해 모델 저장 및 불러오기 기능 등을 제공한다.  
그리고 데이터 변환 모듈에서는 pipeline, 특징 추출(feature extraction), 데이터 전처리(preprocessing), 차원 축소(dimensionality reduction) 등의 기능을 제공한다.  
또 기본적으로 데이터셋을 제공하고있고 이를 불러와 사용할수 있다.  기본 데이터로는 당뇨병 데이터, 아이리스 데이터, 유방암 데이터가 있다.

![scikit-learn algorithm cheat-sheet](https://scikit-learn.org/stable/_static/ml_map.png)

### 사이킷런 설치

사이킷런은 의존성 라이브러리가 존재한다. numpy 와 scipy이다. 이걸 먼저 설치해줘야 하는데 conda로 설치하면 자동으로 설치해준다. 그럼 conda 명령어로 설치하자.

```shell
conda install scikit-learn
```

설치가 완료되면 import를 해보자
```python3
import sklearn
sklearn.__version__
```


두가지 분류모델을 만들어 볼것이다.  
지도학습 모델인 k 최근접 이웃 분류기 모델이고(k-nearest neighbor classifier)
나머지 하나는 비지도 학습 모델인 k평균 군집화 모델이다.(k-means clustering)
하나의 데이터셋을 사용해서 모델을 적용하여 볼 것이다.

실습 데이터셋은 붗꽃(iris)데이터이다. 라이브러리에 내장된 데이터라 바로 사용할수 있다.
```python3
from sklearn.datasets import load_iris
```
그 다음 변수에 할당하고 데이터가 어떤값으로 구성되어있는지 출력해보자.
```python3
iris_dataset = load_iris()
print(iris_dataset key: {}.format(iris_dataset.keys()))
```


In [2]:
# !conda install scikit-learn -y
from sklearn.datasets import load_iris


iris_dataset = load_iris()
print("iris_dataset key: {}".format(iris_dataset.keys()))

iris_dataset key: dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


-------------------------
키 값을 확인해 보면 'data','target','target_names','DESCR','feature_names'로 구성되어있다. 
먼저 **feature_names** 는 
sepal length, sepal width, petal length, petal width로 구분되어있는데 꽃받침의 길이, 너비, 꽃잎의 길이, 너비의 특징을 나타내고있다. 이 값이 들어있는것이 **data**이다.  
그리고 **target**과 **target_names**는 특징에 따른 종의 구분을 나타내는데 Setosa, Versicolor, Virginica 순으로 0,1,2로 쓴다.  
마지막으로 **DESCR**는 Description 의 약자로써 데이터의 전체적인 요약정보를 나타낸다.

사이킷런을 사용하여 학습데이터와 평가데이터로 쉽게 나눌수 있다. 
```python3
from sklearn.model_selection import train_test_split

train_input, test_input, train_label, test_label = train_test_split(iris_dataset['data'],iris_dataset['target'],test_size=0.25, random_state=42)
```

전체 데이터셋에서 25%를 평가데이터셋으로 쓰는 것을 의미한다. 그리고 random_state는 랜덤 상태로써 똑같은 random_state 값을 가지면 같은 무작위 순서를 가진다.
실제 코드를 실행한 후 각 형태를 비교해보자.

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_label, test_label = train_test_split(iris_dataset['data'],iris_dataset['target'],test_size=0.25, random_state=42)

print("shape of train_input : {}".format(train_input.shape))
print("shape of test_input : {}".format(test_input.shape))
print("shape of train_label : {}".format(train_label.shape))
print("shape of test_label : {}".format(test_label.shape))

shape of train_input : (112, 4)
shape of test_input : (38, 4)
shape of train_label : (112,)
shape of test_label : (38,)


이처럼 평가데이터와 학습데이터가 나뉘지 않은 경우는 학습데이터의 일부를 나눠서 사용한다. 또 평가데이터를 따로 분류해서 검증데이터로 사용하는 경우도 있다.

-------------------

### 1. 사이킷런을 이용한 지도학습

지도학습 모델을 만드는 방법을 보자. 지도학습이란 데이터의 정답이 있는 경우 데이터의 정답을 예측하도록 학습시킨다. 그리고 결과를 가지고 다시 학습을 시키는 방법이다.  

위에서 데이터를 준비하였다면 이제 K-nearest neighbor classifier 방식을 아용해서 분류해보자
K-nearest neighbor classifier 방식이란 예측하고자 하는 데이터에 가장 가까운 거리에 있는 데이터의 라벨은 서로 같다고 예측하는 방법이다. 이 방법은 데이터의 사전 학습이 거의 없는 경우의 분류에 많이 사용한다.


![image.png](./img/k-nearest-neighbor.jpg)

위 방식은 근접 점 3개 k=3으로 했을때는 세모를 라벨로 가지겠지만 근접점 5개 k=5로 하게되면 네모를 라벨로 갖게 된다.

이러한 k-최근접 이웃 분류기는 아래의 특징을 가진다.
- 데이터에 대한 가정이 없어 단순하다.
- 다목적 분류와 회귀에 좋다.
- 높은 메모리를 요구한다.
- k 값이 커지면 계산이 늦어질 수 있다.
- 관련없는 기능의 데이터의 규모에 민감하다.

이를 실습해보자.

In [14]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)  # 위에서 설명한 최근접 데이터 수 k

knn.fit(train_input, train_label)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=1, n_neighbors=1, p=2, weights='uniform')

import numpy as np
new_input=np.array([[6.1,2.8,4.7,1.2]])  # 새로운 데이터 입력.

print( knn.predict(new_input) )  # 예측 결과값 출력

[1]


In [15]:
prediction = knn.predict(test_input)
print(prediction)

print(f'test accuracy {np.mean(prediction==test_label)}')

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0]
test accuracy 1.0


### 2. 사이킷런을 이용한 비지도학습

이번엔 비지도학습 모델을 만들어보자. 비지도학습이란 라벨을 사용하지 않고 만들 수 있는 모델이다. 모델을 통해 문제를 해결하고 싶은데 데이터에 대한 정답이 없는 경우에 적용하기 적합한 모델.  
여러 방법중 k-means clustering(k-평균 군집화)을 써보자  

![image.png](./img/k-means.jpg)

이 알고리즘은 계속해서 반복적으로 수행되는데 우선 k개 만큼의 중심을 임의로 설정한다. 그러고 난 후 모든 데이터를 가장 가까운 중심에 할당하며, 같은 중심에 할당된 데이터들을 하나의 군집으로 판단한다. 각 군집내 데이터들을 가지고 군집을 중심으로 새로 구해서 업데이트한다. 이 과정을 계속 반복한다. 그럼 실습을 해보자


In [38]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=3)

k_means.fit(train_input)            # k-maens는 라벨값을 입력하지 않는 비지도 학습.
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto', 
       random_state=1, tol=0.0001, verbose=0)

print('0 cluster : ',train_label[k_means.labels_ ==0])
print('1 cluster : ',train_label[k_means.labels_ ==1])
print('2 cluster : ',train_label[k_means.labels_ ==2])

0 cluster :  [2 1 1 1 2 1 1 1 1 1 2 1 1 1 2 2 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 2 2 1 2 1]
1 cluster :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2 cluster :  [2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 2 2 2]


0번째 군집에는 라벨1번이 주로 분포되어있는것을 볼수 있고, 1번째 군집에는 라벨 0번 데이터만 군집되어 있다. 그리고 2번째 군집에는 라벨2번이 주로 분포된것을 봐서 각 클러스터의 라벨을 확인해 볼수 있다.  
여기서 random_state를 랜점으로 설정하게 되면 군집의 순서가 바뀔수 있다. 군집의 순서보다 군집의 의미를 파악하면 된다.

In [49]:
import numpy as np
new_input=np.array([[6.1,2.8,4.7,1.2]])  # 새로운 데이터 입력.

print( k_means.predict(new_input)) # 새 데이터 측정

predict_cluster=k_means.predict(test_input)
print(predict_cluster)  # 평가데이터 측정

np_arr = np.array(predict_cluster)
print(np_arr.tolist())
np_arr[np_arr==0],np_arr[np_arr==1],np_arr[np_arr==2] = 3,4,5 # 군집과 라벨을 구분하기 위해서 군집의 숫자를 변경.
np_arr[np_arr==3] = 1
np_arr[np_arr==4] = 0
np_arr[np_arr==5] = 2
predict_label = np_arr.tolist()
print(predict_label)


# 성능 체크
print(f'test accuracy : {np.mean(predict_label == test_label)*100:.2f}%')

[0]
[0 1 2 0 0 1 0 2 0 0 2 1 1 1 1 0 2 0 0 2 1 0 1 2 2 2 2 2 1 1 1 1 0 1 1 0 0
 1]
[0, 1, 2, 0, 0, 1, 0, 2, 0, 0, 2, 1, 1, 1, 1, 0, 2, 0, 0, 2, 1, 0, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]
test accuracy : 94.74%


지도학습보다는 낮지만 그래도 매우 높은 성능을 보여준다. 라벨이 없을때는 비지도학습도 나쁘지 않은 방법이다.

만약 데이터가 수치형이 아니라면 어떻게 해야할까? 이러한 상황에서는 특징추출모듈을 사용해야한다. 한번 알아보자.

### 3. 사이킷런을 이용한 특징 추출
관련 모듈을 보면 
- ConuntVectorizer
- TfidfVectorizer
- HashingVectorizer

CounterVectorizer 는 단순히 횟수를 기준으로 특징을 추출한다.  
Tfidf는 Tf-Idf라는 값을 사용해서 특징을 추출한다. 문서내부의 횟수와 전체 문서중에 몇번나오는지를 파악해 쓰는 방법이다.  
Hashing은 counterVectorizer와 비슷한 방법이지만 해시 함수를 사용하여 실행시간을 크게 줄일수 있다. 크기가 클수록 hashingVectorizer를 사용하는게 효과적이다.

간단하므로 코드로 넘어가겠다.

#### CounterVectorizer

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

text_data =['나는 배고다','내일 점심 뭐 먹지','내일 공부해야겠다.','점심 먹고 공부 해야지']

count_vectorizer = CountVectorizer()
count_vectorizer.fit(text_data)
print(count_vectorizer.vocabulary_)

{'나는': 2, '배고다': 6, '내일': 3, '점심': 7, '먹지': 5, '공부해야겠다': 1, '먹고': 4, '공부': 0, '해야지': 8}


사전형태로 정의된 데이터롤 통해서 벡터를 구성해보자.

In [53]:
sentence = [text_data[0]]
print(count_vectorizer.transform(sentence).toarray())

[[0 0 1 0 0 0 1 0 0]]


간단하지만 단순히 횟수만을 나타내서 사용하므로 의미없지만 자주 사용되는 단어들, 지시대명사, 조사 등이 의미있다고 나오는 단점이 있음.

---------------------------------

#### TfidfVectorizer
Tf란 특정단어가 하나의 문서안에서 등장하는 횟수를 말한다. Dfㄴ느 문서 빈도값으로 특정단어가 여러 문서에서 얼마나 자주 등장하는지를 알려주는 것이고 idf란 df값에 역수를 취해 구할수 있다. 이를 통해 CounterVectorizer의 문제를 해결할수 있다.

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

text_data =['나는 배고다','내일 점심 뭐 먹지','내일 공부해야겠다.','점심 먹고 공부 해야지']
tfidf_vectorizer=TfidfVectorizer()

tfidf_vectorizer.fit(text_data)
print(tfidf_vectorizer.vocabulary_)

sentence = [text_data[3]] # 점심먹고 공부 해야지
print(tfidf_vectorizer.transform(sentence).toarray())

{'나는': 2, '배고다': 6, '내일': 3, '점심': 7, '먹지': 5, '공부해야겠다': 1, '먹고': 4, '공부': 0, '해야지': 8}
[[0.52547275 0.         0.         0.         0.52547275 0.
  0.         0.41428875 0.52547275]]
